In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [2]:
IMAGE_SIZE = [350, 350]

train_path = '/kaggle/input/binary-dataset/binary_dataset-augmented/train_dataset/'
valid_path = '/kaggle/input/binary-dataset/binary_dataset-augmented/train_dataset/'

In [3]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


58889256/58889256 [==============================] - 0s 0us/step


In [4]:
for layer in vgg.layers:
    layer.trainable = False

In [5]:
folders = glob('/kaggle/input/binary-dataset/binary_dataset-augmented/train_dataset/*')


In [6]:
x = Flatten()(vgg.output)


In [7]:
prediction = Dense(1, activation='sigmoid')(x)

prediction

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense')>

In [8]:
model = Model(inputs=vgg.input, outputs=prediction)


In [9]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 350, 350, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 350, 350, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 350, 350, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 175, 175, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 175, 175, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 175, 175, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 87, 87, 128)       0     

In [10]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [12]:
base_dir = '/kaggle/input/binary-dataset/binary_dataset_augmented/train_dataset'

training_set = train_datagen.flow_from_directory(base_dir,
                                                         target_size=(350, 350),
                                                         batch_size=32,
                                                         class_mode='binary')

Found 2168 images belonging to 2 classes.


In [13]:
import glob


r = model.fit_generator(
  training_set,
#   validation_data=valid_set,
  epochs=10,
  steps_per_epoch=len(training_set),
#   validation_steps=len(valid_set)
)
   

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/10
68/68 [==============================] - 100s 1s/step - loss: 0.7128 - accuracy: 0.6527
Epoch 2/10
68/68 [==============================] - 69s 1s/step - loss: 0.5157 - accuracy: 0.7288
Epoch 3/10
68/68 [==============================] - 70s 1s/step - loss: 0.4500 - accuracy: 0.7749
Epoch 4/10
68/68 [==============================] - 70s 1s/step - loss: 0.3797 - accuracy: 0.8256
Epoch 5/10
68/68 [==============================] - 69s 1s/step - loss: 0.3415 - accuracy: 0.8450
Epoch 6/10
68/68 [==============================] - 69s 1s/step - loss: 0.3109 - accuracy: 0.8644
Epoch 7/10
68/68 [==============================] - 70s 1s/step - loss: 0.3125 - accuracy: 0.8616
Epoch 8/10
68/68 [==============================] - 70s 1s/step - loss: 0.2774 - accuracy: 0.8870
Epoch 9/10
68/68 [==============================] - 70s 1s/step - loss: 0.2890 - accuracy: 0.8699
Epoch 10/10
68/68 [==============================] - 69s 1s/step - loss: 0.2509 - accuracy: 0.8985


In [14]:

import tensorflow as tf

from keras.models import load_model

model.save('model_vgg16_binary.h5')

In [15]:
from keras.models import load_model

model = load_model('/kaggle/working/model_vgg16_binary.h5')

In [16]:
from keras.preprocessing.image import ImageDataGenerator


test_datagen = ImageDataGenerator(rescale=1./255)

In [17]:
test_set = test_datagen.flow_from_directory('/kaggle/input/binary-dataset/binary_dataset_augmented/test_dataset',
                                             target_size=(350, 350),
                                             batch_size=32,
                                             class_mode='binary')

Found 381 images belonging to 2 classes.


In [18]:
loss, accuracy = model.evaluate(test_set)

print('Test set accuracy:', accuracy)

12/12 [==============================] - 16s 1s/step - loss: 0.3553 - accuracy: 0.8215
Test set accuracy: 0.8215222954750061
